In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(1, '/home/federico/Università/Tesi_magistrale/my_programs/')

from contact_matrix_utils import *
#from src.interchromosomal_cosegregation import *
from numpy.random import Generator, MT19937
from scipy.stats import pearsonr, spearmanr
from scipy.spatial.distance import squareform
from src.utilities import *
import warnings

import seaborn as sns

In [2]:
res = 150E3

In [3]:
save_folder = "/home/federico/Università/Tesi_magistrale/my_programs/interSLICE/Silvia_CryoFISH/"

In [4]:
def select_chromosomes_coordinates(chrA, chrB, chr_windows_sizes):
    if(chrA==0 or chrB ==0):
        print("0 is not accepted as chromosome index ")
        return

    chrA_idx, chrB_idx = chrA - 1, chrB - 1

    if(chrA_idx>chrB_idx):
        chrA_idx, chrB_idx = chrB_idx, chrA_idx

    start_row    = np.sum(chr_windows_sizes[:chrA_idx])
    stop_row     = start_row + chr_windows_sizes[chrA_idx]
    start_column = np.sum(chr_windows_sizes[:chrB_idx])
    stop_column  = start_column + chr_windows_sizes[chrB_idx]

    return start_row, stop_row, start_column, stop_column
which_beta = "$\\beta$ evaluated | 1/2"

# Definition of loci coordinates #

In [5]:
SE17B     = [ 29072733, 29222726]
SE17A     = [ 29281486, 29474597]
_14mTAD1  = [ 50778684, 50955510]
_14hTAD2  = [ 55550566, 55711256]
_14hTAD3  = [ 55770470, 55953023]
_14mTAD4  = [ 67654047, 67922353]
_14SETAD5 = [ 70509172, 70673422]


In [6]:
print(29072733 -  29222726)
print(29281486 -  29474597)
print(50778684 -  50955510)
print(55550566 -  55711256)
print(55770470 -  55953023)
print(67654047 -  67922353)
print(70509172 -  70673422)

-149993
-193111
-176826
-160690
-182553
-268306
-164250


# Import coseg #

In [7]:
name_root_1NP_150kb = "mesc_46C_150kb_481x1"

path_1 =      r"/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/" + name_root_1NP_150kb + "/"
file_name_1 = "segregation_" + name_root_1NP_150kb +  ".pkl"
with open(path_1 + file_name_1, 'rb') as f:
    segregation_table_1NP_150kb = pickle.load(f)

start_chr, stop_chr = 1, 19
stop_chr += 1
chromosomes = []
n_chromosomes = stop_chr - start_chr

l = []
size = 0

for i in range(start_chr, stop_chr):
    chrA = 'chr' + str(i)
    chromosomes += [chrA]
    s = segregation_table_1NP_150kb[chrA]['segregation_table'].shape[0]
    size += s
    l.append(s) 

chr_windows_sizes = np.array(l)

In [8]:
chr14_1NP_150kb_coseg = compute_tube_cosegregation_matrix(segregation_table_1NP_150kb['chr14']['segregation_table'])
chr17_1NP_150kb_coseg = compute_tube_cosegregation_matrix(segregation_table_1NP_150kb['chr17']['segregation_table'])
chr14_17_1NP_150kb_coseg = compute_tube_cosegregation_matrix_offdiag(segregation_table_1NP_150kb['chr14']['segregation_table'], segregation_table_1NP_150kb['chr17']['segregation_table'])

chr14_1NP_150kb_coseg[chr14_1NP_150kb_coseg == 0] = np.nan
chr17_1NP_150kb_coseg[chr17_1NP_150kb_coseg == 0] = np.nan
chr14_17_1NP_150kb_coseg[chr14_17_1NP_150kb_coseg == 0] = np.nan

In [9]:
chr14_17_WDF_1NP_150kb = np.array([])

segregation_table_14 = segregation_table_1NP_150kb["chr14"]["segregation_table"]
chr14_WDF_1NP_150kb = compute_tube_segregation_frequency(segregation_table_14)

segregation_table_17 = segregation_table_1NP_150kb["chr17"]["segregation_table"]
chr17_WDF_1NP_150kb = compute_tube_segregation_frequency(segregation_table_17)

chr14_17_WDF_1NP_150kb = np.concatenate((chr14_WDF_1NP_150kb, chr17_WDF_1NP_150kb))

del segregation_table_14, segregation_table_17

# Import $\pi$ #

In [10]:
chr14_1NP_150kb_pi = squareform(np.loadtxt("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/mesc_46C_150kb_481x1/PI2/PI2_chr14_mesc_46C_150kb_481x1.txt"))
chr17_1NP_150kb_pi = squareform(np.loadtxt("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/mesc_46C_150kb_481x1/PI2/PI2_chr17_mesc_46C_150kb_481x1.txt"))

chr14_1NP_150kb_sign_pi = squareform(np.loadtxt("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/mesc_46C_150kb_481x1/PI2/PI2_significant_95_chr14_mesc_46C_150kb_481x1.txt"))
chr17_1NP_150kb_sign_pi = squareform(np.loadtxt("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/mesc_46C_150kb_481x1/PI2/PI2_significant_95_chr17_mesc_46C_150kb_481x1.txt"))

for i in [chr14_1NP_150kb_pi, chr17_1NP_150kb_pi, chr14_1NP_150kb_sign_pi, chr17_1NP_150kb_sign_pi]:
    np.fill_diagonal(i, np.nan)

In [11]:
chr14_17_1NP_150kb_pi  = np.load("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/mesc_46C_150kb_481x1/PI2_inter_beta_evaluation/PI2_inter_chr14_chr17_mesc_46C_150kb_481x1.npy")
chr14_17_1NP_150kb_sign_pi  = np.load("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/mesc_46C_150kb_481x1/PI2_inter_beta_evaluation/PI2_inter_significant_95_chr14_chr17_mesc_46C_150kb_481x1.npy")


# Heatmaps #

## pi ##

### Full chromosomes ###

In [12]:
fig1, ax1, _, _ = plot_heatmap(chr14_17_1NP_150kb_pi, x_figsize=10, y_figsize=10,colormap="RdYlBu_r", title = "chr 14 17 1NP pi ", resolution=res)

draw_square( SE17A, _14mTAD1, resolution = res,ax = ax1,  x_annotation = "SE17A", y_annotation = "_14mTAD1", edgecolor= 'black', linewidth= 0.1)
draw_square( SE17A, _14hTAD2, resolution = res,ax = ax1,  x_annotation = "SE17A", y_annotation = "_14hTAD2", edgecolor= 'black', linewidth= 0.1)
draw_square( SE17A, _14hTAD3, resolution = res,ax = ax1,  x_annotation = "SE17A", y_annotation = "_14hTAD3", edgecolor= 'black', linewidth= 0.1)
draw_square( SE17A, _14mTAD4, resolution = res,ax = ax1,  x_annotation = "SE17A", y_annotation = "_14mTAD4", edgecolor= 'black', linewidth= 0.1)
draw_square( SE17A, _14SETAD5, resolution = res,ax = ax1, x_annotation = "SE17A", y_annotation = "_14SETAD5", edgecolor='black', linewidth= 0.1)

draw_square( SE17B, _14mTAD1, resolution = res,ax = ax1,  x_annotation = "SE17B", y_annotation = "_14mTAD1", edgecolor= 'black', linewidth= 0.1)
draw_square( SE17B, _14hTAD2, resolution = res,ax = ax1,  x_annotation = "SE17B", y_annotation = "_14hTAD2", edgecolor= 'black', linewidth= 0.1)
draw_square( SE17B, _14hTAD3, resolution = res,ax = ax1,  x_annotation = "SE17B", y_annotation = "_14hTAD3", edgecolor= 'black', linewidth= 0.1)
draw_square( SE17B, _14mTAD4, resolution = res,ax = ax1,  x_annotation = "SE17B", y_annotation = "_14mTAD4", edgecolor= 'black', linewidth= 0.1)
draw_square( SE17B, _14SETAD5, resolution = res,ax = ax1, x_annotation = "SE17B", y_annotation = "_14SETAD5", edgecolor='black', linewidth= 0.1)

fig1.savefig(save_folder + "heatmaps/inter_pi/" +  'chr14_17_1NP_150kb_pi.svg' , format = 'svg', dpi = 300)

plt.close(fig1)

del fig1

In [13]:
fig1, ax1, _, _ = plot_heatmap(chr14_17_1NP_150kb_sign_pi,x_figsize=10, y_figsize=10, colormap="RdYlBu_r", title = "chr 14 17 1NP sign pi", resolution=res)

draw_square( SE17A, _14mTAD1, resolution = res,ax = ax1,  x_annotation = "SE17A", y_annotation = "_14mTAD1", edgecolor= 'black', linewidth= 0.1)
draw_square( SE17A, _14hTAD2, resolution = res,ax = ax1,  x_annotation = "SE17A", y_annotation = "_14hTAD2", edgecolor= 'black', linewidth= 0.1)
draw_square( SE17A, _14hTAD3, resolution = res,ax = ax1,  x_annotation = "SE17A", y_annotation = "_14hTAD3", edgecolor= 'black', linewidth= 0.1)
draw_square( SE17A, _14mTAD4, resolution = res,ax = ax1,  x_annotation = "SE17A", y_annotation = "_14mTAD4", edgecolor= 'black', linewidth= 0.1)
draw_square( SE17A, _14SETAD5, resolution = res,ax = ax1, x_annotation = "SE17A", y_annotation = "_14SETAD5", edgecolor='black', linewidth= 0.1)

draw_square( SE17B, _14mTAD1, resolution = res,ax = ax1,  x_annotation = "SE17B", y_annotation = "_14mTAD1", edgecolor= 'black', linewidth= 0.1)
draw_square( SE17B, _14hTAD2, resolution = res,ax = ax1,  x_annotation = "SE17B", y_annotation = "_14hTAD2", edgecolor= 'black', linewidth= 0.1)
draw_square( SE17B, _14hTAD3, resolution = res,ax = ax1,  x_annotation = "SE17B", y_annotation = "_14hTAD3", edgecolor= 'black', linewidth= 0.1)
draw_square( SE17B, _14mTAD4, resolution = res,ax = ax1,  x_annotation = "SE17B", y_annotation = "_14mTAD4", edgecolor= 'black', linewidth= 0.1)
draw_square( SE17B, _14SETAD5, resolution = res,ax = ax1, x_annotation = "SE17B", y_annotation = "_14SETAD5", edgecolor='black', linewidth= 0.1)

fig1.savefig(save_folder + "heatmaps/inter_sign_pi/" +  'chr14_17_1NP_150kb_sign_pi.svg' , format = 'svg', dpi = 300)

plt.close(fig1)

del fig1

### zoom ###

In [14]:
region_1 = np.array([[ int((SE17B[0])/res)-10 , int((SE17A[1])/res) + 10 ], [int((_14mTAD1[0])/res)-10 , int((_14hTAD3[1])/res) + 10 ]])
region_2 = np.array([[ int((SE17B[0])/res)-10 , int((SE17A[1])/res) + 10 ], [int((_14mTAD4[0])/res)-10 , int((_14SETAD5[1])/res) + 10 ]])

In [19]:
fig1, ax1, _, _ = plot_heatmap(chr14_17_1NP_150kb_pi[region_1[1,0]:region_1[1,1], region_1[0,0]:region_1[0,1]], colormap = "RdYlBu_r", title = "chr 14 17 pi 1NP")

draw_square( SE17A, _14mTAD1, resolution = res,ax = ax1,  x_annotation = "SE17A", y_annotation = "_14mTAD1", edgecolor= 'black', linewidth= 1, x_s=region_1[0,0], y_s=region_1[1,0])
draw_square( SE17A, _14hTAD2, resolution = res,ax = ax1,  x_annotation = "SE17A", y_annotation = "_14hTAD2", edgecolor= 'black', linewidth= 1, x_s=region_1[0,0], y_s=region_1[1,0])
draw_square( SE17A, _14hTAD3, resolution = res,ax = ax1,  x_annotation = "SE17A", y_annotation = "_14hTAD3", edgecolor= 'black', linewidth= 1, x_s=region_1[0,0], y_s=region_1[1,0])
draw_square( SE17A, _14mTAD4, resolution = res,ax = ax1,  x_annotation = "SE17A", y_annotation = "_14mTAD4", edgecolor= 'black', linewidth= 1, x_s=region_1[0,0], y_s=region_1[1,0])
draw_square( SE17A, _14SETAD5, resolution = res,ax = ax1, x_annotation = "SE17A", y_annotation = "_14SETAD5", edgecolor='black', linewidth= 1, x_s=region_1[0,0], y_s=region_1[1,0])

draw_square( SE17B, _14mTAD1, resolution = res,ax = ax1,  x_annotation = "SE17B", y_annotation = "_14mTAD1", edgecolor= 'black', linewidth= 1, x_s=region_1[0,0], y_s=region_1[1,0])
draw_square( SE17B, _14hTAD2, resolution = res,ax = ax1,  x_annotation = "SE17B", y_annotation = "_14hTAD2", edgecolor= 'black', linewidth= 1, x_s=region_1[0,0], y_s=region_1[1,0])
draw_square( SE17B, _14hTAD3, resolution = res,ax = ax1,  x_annotation = "SE17B", y_annotation = "_14hTAD3", edgecolor= 'black', linewidth= 1, x_s=region_1[0,0], y_s=region_1[1,0])
draw_square( SE17B, _14mTAD4, resolution = res,ax = ax1,  x_annotation = "SE17B", y_annotation = "_14mTAD4", edgecolor= 'black', linewidth= 1, x_s=region_1[0,0], y_s=region_1[1,0])
draw_square( SE17B, _14SETAD5, resolution = res,ax = ax1, x_annotation = "SE17B", y_annotation = "_14SETAD5", edgecolor='black', linewidth= 1, x_s=region_1[0,0], y_s=region_1[1,0])

fig1.savefig(save_folder + "heatmaps/detail/inter_pi/"  +  'chr14_17_1NP_150kb_pi_region_1.svg' , format = 'svg', dpi = 300)

plt.close(fig1)

del fig1

In [20]:
fig1, ax1, _, _ = plot_heatmap(chr14_17_1NP_150kb_pi[region_2[1,0]:region_2[1,1], region_2[0,0]:region_2[0,1]], colormap = "RdYlBu_r", title = "chr 14 17 pi 1NP")

draw_square( SE17A, _14mTAD1, resolution = res,ax = ax1,  x_annotation = "SE17A", y_annotation = "_14mTAD1", edgecolor= 'black', linewidth= 1, x_s=region_2[0,0], y_s=region_2[1,0])
draw_square( SE17A, _14hTAD2, resolution = res,ax = ax1,  x_annotation = "SE17A", y_annotation = "_14hTAD2", edgecolor= 'black', linewidth= 1, x_s=region_2[0,0], y_s=region_2[1,0])
draw_square( SE17A, _14hTAD3, resolution = res,ax = ax1,  x_annotation = "SE17A", y_annotation = "_14hTAD3", edgecolor= 'black', linewidth= 1, x_s=region_2[0,0], y_s=region_2[1,0])
draw_square( SE17A, _14mTAD4, resolution = res,ax = ax1,  x_annotation = "SE17A", y_annotation = "_14mTAD4", edgecolor= 'black', linewidth= 1, x_s=region_2[0,0], y_s=region_2[1,0])
draw_square( SE17A, _14SETAD5, resolution = res,ax = ax1, x_annotation = "SE17A", y_annotation = "_14SETAD5", edgecolor='black', linewidth= 1, x_s=region_2[0,0], y_s=region_2[1,0])

draw_square( SE17B, _14mTAD1, resolution = res,ax = ax1,  x_annotation = "SE17B", y_annotation = "_14mTAD1", edgecolor= 'black', linewidth= 1, x_s=region_2[0,0], y_s=region_2[1,0])
draw_square( SE17B, _14hTAD2, resolution = res,ax = ax1,  x_annotation = "SE17B", y_annotation = "_14hTAD2", edgecolor= 'black', linewidth= 1, x_s=region_2[0,0], y_s=region_2[1,0])
draw_square( SE17B, _14hTAD3, resolution = res,ax = ax1,  x_annotation = "SE17B", y_annotation = "_14hTAD3", edgecolor= 'black', linewidth= 1, x_s=region_2[0,0], y_s=region_2[1,0])
draw_square( SE17B, _14mTAD4, resolution = res,ax = ax1,  x_annotation = "SE17B", y_annotation = "_14mTAD4", edgecolor= 'black', linewidth= 1, x_s=region_2[0,0], y_s=region_2[1,0])
draw_square( SE17B, _14SETAD5, resolution = res,ax = ax1, x_annotation = "SE17B", y_annotation = "_14SETAD5", edgecolor='black', linewidth= 1, x_s=region_2[0,0], y_s=region_2[1,0])

fig1.savefig(save_folder + "heatmaps/detail/inter_pi/"  +  'chr14_17_1NP_150kb_pi_region_2.svg' , format = 'svg', dpi = 300)

plt.close(fig1)

del fig1